<a href="https://colab.research.google.com/github/ugywjd/pancancer/blob/master/phj_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os

In [74]:
CCLE_expression = pd.read_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/CCLE_expression.csv')
cellline_TCGA = pd.read_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/CCLE_TCGA.csv')

In [4]:
#결측치 제거
cellline_TCGA = cellline_TCGA.dropna(axis = 0)
#cellline_TCGA.isnull().sum()

In [5]:
#GBM, LGG cellline만 가져오기
#cellline_TCGA = cellline_TCGA[(cellline_TCGA['TCGA']=='GBM')|(cellline_TCGA['TCGA']=='LGG')]

In [76]:
cellline_TCGA

,DepMap_ID,CELLLINE,CCLE,TCGA
0,ACH-000288,BT549_BREAST,BT-549,BRCA
1,ACH-000009,C2BBE1_LARGE_INTESTINE,C2BBe1,COADREAD
2,ACH-000018,T24_URINARY_TRACT,T24,BLCA
3,ACH-000019,MCF7_BREAST,MCF7,BRCA
4,ACH-000021,NCIH1693_LUNG,NCI-H1693,LUAD
...,...,...,...,...
1835,ACH-002391,UDSCC2_UPPER_AERODIGESTIVE_TRACT,UD-SCC-2,NaN
1836,ACH-002393,CROAP3_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,CRO-AP3,NaN
1837,ACH-002394,GEO_LARGE_INTESTINE,GEO,NaN
1838,ACH-002395,HUH6CLONE5_LIVER,HuH-6 Clone 5,NaN


In [77]:
#gdsc2 data
drug_reference_2 = pd.read_excel('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/GDSC2_fitted_dose_response_24Jul22.xlsx')
drug_reference_2

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-1.462148,0.930105,0.088999,0.432482
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-4.869447,0.614932,0.111423,-1.420322
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.360684,0.790953,0.142754,-0.599894
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-5.045014,0.592624,0.135642,-1.515791
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.741620,0.733992,0.128066,-0.807038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242031,GDSC2,401,19109764,930083,RO82-W-1,SIDM00218,THCA,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,11.012365,0.922586,0.058611,1.165681
242032,GDSC2,401,19110496,930299,TT,SIDM00484,THCA,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.114089,0.910080,0.047599,0.136568
242033,GDSC2,401,19130352,1240154,IHH-4,SIDM00600,THCA,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.235088,0.981456,0.079003,0.275191
242034,GDSC2,401,19142997,1240223,TT2609-C02,SIDM00418,THCA,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,9.661615,0.913009,0.078170,-0.381810


In [78]:
#cellline, gdsc2 합치기
cell_gdsc = pd.merge(cellline_TCGA, drug_reference_2, left_on='CCLE', right_on='CELL_LINE_NAME')
cell_gdsc

,DepMap_ID,CELLLINE,CCLE,TCGA,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,...,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,ACH-000288,BT549_BREAST,BT-549,BRCA,GDSC2,401,18991930,905951,BT-549,SIDM00122,...,TOP1,DNA replication,1046,Y,0.000098,0.1000,-0.846118,0.915100,0.071449,0.767464
1,ACH-000288,BT549_BREAST,BT-549,BRCA,GDSC2,401,18991931,905951,BT-549,SIDM00122,...,Microtubule destabiliser,Mitosis,1046,Y,0.000100,0.1000,-2.182955,0.841960,0.117882,0.601852
2,ACH-000288,BT549_BREAST,BT-549,BRCA,GDSC2,401,18991932,905951,BT-549,SIDM00122,...,DNA crosslinker,DNA replication,1046,Y,0.003906,8.0000,5.729519,0.978570,0.063031,1.344217
3,ACH-000288,BT549_BREAST,BT-549,BRCA,GDSC2,401,18991933,905951,BT-549,SIDM00122,...,Antimetabolite,Other,1046,Y,0.002001,2.0000,2.807780,0.938989,0.052028,0.549474
4,ACH-000288,BT549_BREAST,BT-549,BRCA,GDSC2,401,18991934,905951,BT-549,SIDM00122,...,Microtubule stabiliser,Mitosis,1046,Y,0.000012,0.0125,-3.855598,0.868214,0.070789,0.319116
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198051,ACH-002223,D245MG_CENTRAL_NERVOUS_SYSTEM,D-245MG,GBM,GDSC2,401,19113316,946366,D-245MG,SIDM01021,...,"BRD2, BRD3, BRD4, BRDT",Chromatin other,1046,Y,0.010005,10.0000,2.312935,0.837735,0.077087,0.147166
198052,ACH-002223,D245MG_CENTRAL_NERVOUS_SYSTEM,D-245MG,GBM,GDSC2,401,19113317,946366,D-245MG,SIDM01021,...,BRD4,Chromatin other,1046,Y,0.010005,10.0000,6.197481,0.978331,0.038525,1.902977
198053,ACH-002223,D245MG_CENTRAL_NERVOUS_SYSTEM,D-245MG,GBM,GDSC2,401,19113318,946366,D-245MG,SIDM01021,...,EGLN1,Other,1046,Y,0.000500,0.5000,3.205445,0.978377,0.038633,0.992504
198054,ACH-002223,D245MG_CENTRAL_NERVOUS_SYSTEM,D-245MG,GBM,GDSC2,401,19113319,946366,D-245MG,SIDM01021,...,"GSK3A, GSK3B",WNT signaling,1046,Y,0.010005,10.0000,4.054767,0.929422,0.078763,-0.392293


In [79]:
### drug name normalization
drug_reference_1 = pd.read_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/drug_response_predict/raw_data/DrugCorrection1.csv')
drug_reference_2 = pd.read_excel('/content/drive/MyDrive/Hong_Lab/21. Depmap/drug_response_predict/raw_data/bioinfo16_supplementary_tables.xlsx', sheet_name=1, header=2).drop('DrugBank ID', axis=1)
drug_standard = cell_gdsc.merge(drug_reference_1, how = 'left', left_on = 'DRUG_NAME', right_on = 'OldName').drop('OldName', axis=1)
drug_standard = cell_gdsc.merge(drug_reference_2, how = 'left', left_on = 'DRUG_NAME', right_on = 'Recorded Name from TCGA').drop('Recorded Name from TCGA', axis=1)

In [10]:
drug_standard

,DepMap_ID,CCLE,TCGA,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,...,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE,Standard drug name
0,ACH-000288,BT-549,BRCA,GDSC2,401,18991930,905951,BT-549,SIDM00122,BRCA,...,DNA replication,1046,Y,0.000098,0.1000,-0.846118,0.915100,0.071449,0.767464,NaN
1,ACH-000288,BT-549,BRCA,GDSC2,401,18991931,905951,BT-549,SIDM00122,BRCA,...,Mitosis,1046,Y,0.000100,0.1000,-2.182955,0.841960,0.117882,0.601852,Vinblastine
2,ACH-000288,BT-549,BRCA,GDSC2,401,18991932,905951,BT-549,SIDM00122,BRCA,...,DNA replication,1046,Y,0.003906,8.0000,5.729519,0.978570,0.063031,1.344217,Cisplatin
3,ACH-000288,BT-549,BRCA,GDSC2,401,18991933,905951,BT-549,SIDM00122,BRCA,...,Other,1046,Y,0.002001,2.0000,2.807780,0.938989,0.052028,0.549474,NaN
4,ACH-000288,BT-549,BRCA,GDSC2,401,18991934,905951,BT-549,SIDM00122,BRCA,...,Mitosis,1046,Y,0.000012,0.0125,-3.855598,0.868214,0.070789,0.319116,Docetaxel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164077,ACH-002223,D-245MG,GBM,GDSC2,401,19113316,946366,D-245MG,SIDM01021,GBM,...,Chromatin other,1046,Y,0.010005,10.0000,2.312935,0.837735,0.077087,0.147166,NaN
164078,ACH-002223,D-245MG,GBM,GDSC2,401,19113317,946366,D-245MG,SIDM01021,GBM,...,Chromatin other,1046,Y,0.010005,10.0000,6.197481,0.978331,0.038525,1.902977,NaN
164079,ACH-002223,D-245MG,GBM,GDSC2,401,19113318,946366,D-245MG,SIDM01021,GBM,...,Other,1046,Y,0.000500,0.5000,3.205445,0.978377,0.038633,0.992504,NaN
164080,ACH-002223,D-245MG,GBM,GDSC2,401,19113319,946366,D-245MG,SIDM01021,GBM,...,WNT signaling,1046,Y,0.010005,10.0000,4.054767,0.929422,0.078763,-0.392293,NaN


In [80]:
drug_standard_Temozolomide = drug_standard[(drug_standard['DRUG_NAME']=='Temozolomide')]

In [81]:
drug_standard_Temozolomide.to_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/cell_gdsc_Temozolomide.csv',index=False)

In [13]:
drug_standard.isnull().sum() #Standard drug name 7663

DepMap_ID                  0
CCLE                       0
TCGA                       0
DATASET                    0
NLME_RESULT_ID             0
NLME_CURVE_ID              0
COSMIC_ID                  0
CELL_LINE_NAME             0
SANGER_MODEL_ID            0
TCGA_DESC                180
DRUG_ID                    0
DRUG_NAME                  0
PUTATIVE_TARGET        18149
PATHWAY_NAME               0
COMPANY_ID                 0
WEBRELEASE                 0
MIN_CONC                   0
MAX_CONC                   0
LN_IC50                    0
AUC                        0
RMSE                       0
Z_SCORE                    0
Standard drug name    145079
dtype: int64

In [82]:
drug_standard['measure_of_response'] = drug_standard['Z_SCORE']
drug_standard = drug_standard[(drug_standard['measure_of_response'] >= 2 )|(drug_standard['measure_of_response'] <= -2)]

In [83]:
drug_standard.loc[drug_standard['measure_of_response'] > 2, 'measure_of_response'] = 'Resistant'
drug_standard.loc[drug_standard['Z_SCORE'] < -2, 'measure_of_response'] = 'Sensitive'
drug_standard

,DepMap_ID,CELLLINE,CCLE,TCGA,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,...,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE,Standard drug name,measure_of_response
78,ACH-000288,BT549_BREAST,BT-549,BRCA,GDSC2,401,18992008,905951,BT-549,SIDM00122,...,1046,Y,0.000100,1.0,2.747072,0.987950,0.021459,2.168307,NaN,Resistant
172,ACH-000288,BT549_BREAST,BT-549,BRCA,GDSC2,401,18992103,905951,BT-549,SIDM00122,...,1043,Y,0.020011,20.0,0.296126,0.583330,0.114137,-2.303944,NaN,Sensitive
208,ACH-000288,BT549_BREAST,BT-549,BRCA,GDSC2,401,18992140,905951,BT-549,SIDM00122,...,1046,Y,0.010005,10.0,6.118141,0.975722,0.058101,2.329971,NaN,Resistant
318,ACH-000009,C2BBE1_LARGE_INTESTINE,C2BBe1,COADREAD,GDSC2,401,19086959,910700,C2BBe1,SIDM01233,...,1046,Y,0.010005,10.0,6.392406,0.967399,0.054059,2.236524,NaN,Resistant
339,ACH-000009,C2BBE1_LARGE_INTESTINE,C2BBe1,COADREAD,GDSC2,401,19086980,910700,C2BBe1,SIDM01233,...,1046,Y,0.001001,10.0,4.365715,0.971441,0.030317,2.048748,NaN,Resistant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197817,ACH-002204,UCH2_BONE,U-CH2,NaN,GDSC2,401,19184746,1503373,U-CH2,SIDM01185,...,1046,Y,0.020011,20.0,6.957933,0.978926,0.059935,2.294262,NaN,Resistant
197872,ACH-002204,UCH2_BONE,U-CH2,NaN,GDSC2,401,19184801,1503373,U-CH2,SIDM01185,...,1046,Y,0.010005,10.0,6.461130,0.981175,0.036350,2.137098,NaN,Resistant
197901,ACH-002223,D245MG_CENTRAL_NERVOUS_SYSTEM,D-245MG,GBM,GDSC2,401,19113166,946366,D-245MG,SIDM01021,...,1046,Y,0.007813,2.0,-5.601285,0.140967,0.088340,-2.027417,NaN,Sensitive
197911,ACH-002223,D245MG_CENTRAL_NERVOUS_SYSTEM,D-245MG,GBM,GDSC2,401,19113176,946366,D-245MG,SIDM01021,...,1046,Y,0.010005,10.0,0.899400,0.719467,0.110431,-2.037714,NaN,Sensitive


In [84]:
drug_standard.to_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/cell_gdsc_res_sen.csv',index=False)

In [19]:
from sklearn.preprocessing import MultiLabelBinarizer

### MultiLabelBinarization을 위한 class list 생성
classes = set(drug_standard['DRUG_NAME'].to_list())

In [20]:
### MultiLabelBinarization
mlb = MultiLabelBinarizer()
mlb.fit([classes])#이건 아무것도 안한데 아래 transform만 있어도 돌아는 가는 것 같아 

tmp = drug_standard['DRUG_NAME'].str.split(', ').to_list()
tmp_result = mlb.transform(tmp)

df_mlb = pd.DataFrame(tmp_result, columns = mlb.classes_)#약물 one hot code
df_mlb

,123138,123829,150412,5-Fluorouracil,5-azacytidine,50869,615590,630600,667880,720427,...,WZ4003,Wee1 Inhibitor,Wnt-C59,XAV939,YK-4-279,ZM447439,Zoledronate,alpha-lipoic acid,ascorbate (vitamin C),glutathione
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7823,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7824,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7825,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7826,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_mlb.sum()

123138                   25
123829                    8
150412                   29
5-Fluorouracil           26
5-azacytidine            20
                         ..
ZM447439                 27
Zoledronate              20
alpha-lipoic acid        27
ascorbate (vitamin C)    38
glutathione              24
Length: 286, dtype: int64

In [85]:
response_dataset = pd.concat([drug_standard[['DepMap_ID','CELLLINE','CCLE', 'DRUG_NAME']], df_mlb, drug_standard['measure_of_response']], axis = 1)

In [23]:
response_dataset

,DepMap_ID,CCLE,DRUG_NAME,123138,123829,150412,5-Fluorouracil,5-azacytidine,50869,615590,...,Wee1 Inhibitor,Wnt-C59,XAV939,YK-4-279,ZM447439,Zoledronate,alpha-lipoic acid,ascorbate (vitamin C),glutathione,measure_of_response
0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN
4,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163884,ACH-002317,YT,GSK-LSD1-2HCl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sensitive
163894,ACH-002317,YT,GSK2830371,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sensitive
163927,ACH-002223,D-245MG,Staurosporine,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sensitive
163937,ACH-002223,D-245MG,PD173074,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sensitive


##EXPRESSION


In [49]:
a = drug_standard[['DepMap_ID','measure_of_response']]

In [50]:
exp_res = pd.merge(a, CCLE_expression, left_on='DepMap_ID', right_on='DepMap_ID')

In [51]:
exp_res.index = exp_res['DepMap_ID']
exp_res = exp_res.drop(labels=['DepMap_ID'],axis=1)

In [52]:
exp_res

,measure_of_response,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,...,OR8B8,ZNF772,ZNF257,PCNX3,ADAM32,ACSL5,LRRC8B,ABCB8,H3C12,SND1
DepMap_ID,,,,,,,,,,,,,,,,,,,,,
ACH-000288,Resistant,3.390943,0.000000,7.142924,2.169925,4.195348,0.014355,2.356144,6.644722,4.189034,...,0.028569,1.839960,1.985500,4.483493,1.130931,0.807355,1.207893,4.643856,0.575312,6.635464
ACH-000288,Sensitive,3.390943,0.000000,7.142924,2.169925,4.195348,0.014355,2.356144,6.644722,4.189034,...,0.028569,1.839960,1.985500,4.483493,1.130931,0.807355,1.207893,4.643856,0.575312,6.635464
ACH-000288,Resistant,3.390943,0.000000,7.142924,2.169925,4.195348,0.014355,2.356144,6.644722,4.189034,...,0.028569,1.839960,1.985500,4.483493,1.130931,0.807355,1.207893,4.643856,0.575312,6.635464
ACH-000009,Resistant,5.761019,0.111031,8.081031,2.241840,4.448901,0.000000,0.042644,7.130313,5.691255,...,0.000000,0.389567,0.000000,4.493775,0.056584,4.430954,2.253989,4.608809,0.389567,7.549900
ACH-000009,Resistant,5.761019,0.111031,8.081031,2.241840,4.448901,0.000000,0.042644,7.130313,5.691255,...,0.000000,0.389567,0.000000,4.493775,0.056584,4.430954,2.253989,4.608809,0.389567,7.549900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACH-002062,Sensitive,0.014355,0.000000,6.908333,4.029453,3.714795,6.076388,0.000000,4.742006,3.963474,...,0.000000,1.831877,2.195348,3.570463,0.028569,5.377818,1.937344,3.476382,0.163499,7.059399
ACH-002062,Sensitive,0.014355,0.000000,6.908333,4.029453,3.714795,6.076388,0.000000,4.742006,3.963474,...,0.000000,1.831877,2.195348,3.570463,0.028569,5.377818,1.937344,3.476382,0.163499,7.059399
ACH-002062,Sensitive,0.014355,0.000000,6.908333,4.029453,3.714795,6.076388,0.000000,4.742006,3.963474,...,0.000000,1.831877,2.195348,3.570463,0.028569,5.377818,1.937344,3.476382,0.163499,7.059399


In [ ]:
exp_res.dtypes

In [65]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [54]:
#모델 만들기
X = exp_res.drop(['measure_of_response'], axis=1)
y = exp_res['measure_of_response']

In [55]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [56]:
X_train.shape, X_test.shape

((5102, 16382), (1276, 16382))

In [57]:
#Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
# instantiate the classifier 
# n_estimators=10
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
y_pred = rfc.predict(X_test)

In [58]:
from sklearn.metrics import accuracy_score
print('Model accuracy score with 10 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred)))

Model accuracy score with 10 decision-trees : 0.9491


In [61]:
#n_estimators=100 
rfc_100 = RandomForestClassifier(n_estimators=100, random_state=0)
# fit the model to the training set
rfc_100.fit(X_train, y_train)

RandomForestClassifier(n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

y_pred_100 = rfc_100.predict(X_test)

print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred_100))) # 성능 높여도 비슷한듯

Model accuracy score with 100 decision-trees : 0.9491


In [62]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_100))

              precision    recall  f1-score   support

   Resistant       0.95      0.97      0.96       749
   Sensitive       0.95      0.93      0.94       527

    accuracy                           0.95      1276
   macro avg       0.95      0.95      0.95      1276
weighted avg       0.95      0.95      0.95      1276



In [63]:
#Feature importance
clf = RandomForestClassifier(n_estimators=100, random_state=0)
clf.fit(X_train, y_train)
feature_scores = pd.Series(clf.feature_importances_, index=X_train.columns).sort_values(ascending=False)
feature_scores[:10]

SLC49A4     0.027351
WASL        0.019918
SH2D4A      0.015004
KRT7        0.014242
AJUBA       0.014037
GPR39       0.011284
PDGFB       0.010876
CTNNA1      0.010463
WNT7B       0.010212
PERP        0.010049
dtype: float64

In [68]:
#SVM
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print("\nAccuracy:", clf.score(X_test, y_test) * 100)

              precision    recall  f1-score   support

   Resistant       0.92      0.95      0.94       749
   Sensitive       0.93      0.89      0.91       527

    accuracy                           0.93      1276
   macro avg       0.93      0.92      0.92      1276
weighted avg       0.93      0.93      0.93      1276


Accuracy: 92.55485893416929


In [70]:
#XGboost
LE = LabelEncoder()
y_train = LE.fit_transform(y_train)
y_test = LE.fit_transform(y_test)

clf = XGBClassifier(eta = 0.5, max_leaf_nodes = 10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred), "\n")
print("\nAccuracy:", clf.score(X_test, y_test) * 100)

              precision    recall  f1-score   support

           0       0.95      0.97      0.96       749
           1       0.95      0.92      0.94       527

    accuracy                           0.95      1276
   macro avg       0.95      0.95      0.95      1276
weighted avg       0.95      0.95      0.95      1276
 


Accuracy: 94.98432601880877


## RPPA

In [73]:
rppa = pd.read_csv('/content/drive/MyDrive/Hong_Lab/21. Depmap/phj_model/raw_data/CCLE_RPPA_20180123.csv')
#rppa.index.name = None
#rppa = rppa.transpose()
rppa

,Unnamed: 0,14-3-3_beta,14-3-3_epsilon_Caution,14-3-3_zeta,4E-BP1,4E-BP1_pS65,4E-BP1_pT37_T46,4E-BP1_pT70,53BP1,A-Raf_pS299_Caution,...,Tuberin_pT1462,VAV1_Caution,VEGFR2,VHL_Caution,XBP1_Caution,XRCC1_Caution,YAP_Caution,YAP_pS127_Caution,YB-1,YB-1_pS102
0,DMS53_LUNG,-0.104888,0.060414,0.309068,-0.075506,0.230359,0.198304,-0.030541,0.455889,0.090484,...,-0.099433,-0.486715,-1.147858,0.133876,-0.075812,-0.144388,-1.090303,-2.109324,0.178104,0.246541
1,SW1116_LARGE_INTESTINE,0.358504,-0.180291,-0.041237,-0.286629,-0.877406,-1.026948,-0.462761,-0.011197,0.605330,...,-0.109777,0.349330,0.770148,0.984297,-0.168138,-0.004905,0.189294,-0.283593,0.255972,-0.121134
2,NCIH1694_LUNG,0.028738,0.071902,-0.094847,0.285069,1.321551,0.620703,-0.439484,0.195007,0.036221,...,0.154344,-0.478189,-1.185530,1.273013,-0.240413,0.476633,-1.367465,-2.525695,-0.137880,-0.451282
3,P3HR1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,0.120039,-0.066802,-0.128007,-0.552081,-0.292428,-1.415935,-0.138858,-0.066122,-0.346564,...,0.040106,5.923830,-3.893832,-2.499188,0.632758,0.025639,-1.189180,-3.056863,0.025997,-0.465205
4,HUT78_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE,-0.268997,-0.060281,-0.137881,-0.398729,-0.095622,-0.533905,0.054245,-0.573022,-0.162968,...,-0.466919,5.475880,-0.561973,-0.500953,-0.261494,0.358679,-0.951686,-3.247388,-0.151424,-0.145426
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,UO31_KIDNEY,0.043231,0.093158,0.105461,0.011264,-0.264052,-0.079559,-0.000355,-0.308669,-0.025941,...,-0.109095,0.137608,0.618270,-0.541645,0.263695,-0.092934,0.433857,0.327542,0.108921,-0.197684
895,SF268_CENTRAL_NERVOUS_SYSTEM,-0.034597,0.065964,-0.329024,-0.540973,0.492108,0.137051,-0.003109,-0.276884,-0.390067,...,-0.339392,-0.054790,-0.271650,-1.735560,-0.397684,-0.166362,2.550478,3.225039,0.136907,0.426637
896,SF539_CENTRAL_NERVOUS_SYSTEM,-0.250998,0.420490,-0.334213,-0.156368,-0.219208,-0.187704,0.240884,0.352163,0.052992,...,0.094319,0.346651,-0.367292,-1.851276,0.069242,0.165163,-0.174660,-0.811089,0.067923,0.237027
897,SNB75_CENTRAL_NERVOUS_SYSTEM,-0.139833,0.194831,-0.135708,-0.434248,0.208941,-0.071338,0.145042,-0.457499,0.032008,...,0.159241,-0.140213,0.436948,-1.476417,-0.390487,-0.163100,0.407310,0.390911,-0.034470,0.153921


In [89]:
b = drug_standard[['CELLLINE','measure_of_response']]
b

,CELLLINE,measure_of_response
78,BT549_BREAST,Resistant
172,BT549_BREAST,Sensitive
208,BT549_BREAST,Resistant
318,C2BBE1_LARGE_INTESTINE,Resistant
339,C2BBE1_LARGE_INTESTINE,Resistant
...,...,...
197817,UCH2_BONE,Resistant
197872,UCH2_BONE,Resistant
197901,D245MG_CENTRAL_NERVOUS_SYSTEM,Sensitive
197911,D245MG_CENTRAL_NERVOUS_SYSTEM,Sensitive


In [90]:
rppa_res = pd.merge(b, rppa, left_on='CELLLINE', right_on='Unnamed: 0')

In [91]:
rppa_res

,CELLLINE,measure_of_response,Unnamed: 0,14-3-3_beta,14-3-3_epsilon_Caution,14-3-3_zeta,4E-BP1,4E-BP1_pS65,4E-BP1_pT37_T46,4E-BP1_pT70,...,Tuberin_pT1462,VAV1_Caution,VEGFR2,VHL_Caution,XBP1_Caution,XRCC1_Caution,YAP_Caution,YAP_pS127_Caution,YB-1,YB-1_pS102
0,C2BBE1_LARGE_INTESTINE,Resistant,C2BBE1_LARGE_INTESTINE,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
1,C2BBE1_LARGE_INTESTINE,Resistant,C2BBE1_LARGE_INTESTINE,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
2,C2BBE1_LARGE_INTESTINE,Resistant,C2BBE1_LARGE_INTESTINE,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
3,C2BBE1_LARGE_INTESTINE,Sensitive,C2BBE1_LARGE_INTESTINE,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
4,C2BBE1_LARGE_INTESTINE,Resistant,C2BBE1_LARGE_INTESTINE,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6251,NCIH2066_LUNG,Resistant,NCIH2066_LUNG,-0.130911,0.300023,-0.001885,0.130247,0.782626,0.403611,0.087359,...,0.072654,-0.971477,-1.126916,0.595119,0.271774,-0.084982,-1.244111,-2.312100,0.333945,-0.046056
6252,NCIH2066_LUNG,Resistant,NCIH2066_LUNG,-0.130911,0.300023,-0.001885,0.130247,0.782626,0.403611,0.087359,...,0.072654,-0.971477,-1.126916,0.595119,0.271774,-0.084982,-1.244111,-2.312100,0.333945,-0.046056
6253,NCIH2066_LUNG,Resistant,NCIH2066_LUNG,-0.130911,0.300023,-0.001885,0.130247,0.782626,0.403611,0.087359,...,0.072654,-0.971477,-1.126916,0.595119,0.271774,-0.084982,-1.244111,-2.312100,0.333945,-0.046056
6254,NCIH2066_LUNG,Resistant,NCIH2066_LUNG,-0.130911,0.300023,-0.001885,0.130247,0.782626,0.403611,0.087359,...,0.072654,-0.971477,-1.126916,0.595119,0.271774,-0.084982,-1.244111,-2.312100,0.333945,-0.046056


In [92]:
rppa_res.index = rppa_res['CELLLINE']
rppa_res = rppa_res.drop(labels=['CELLLINE'],axis=1)
rppa_res = rppa_res.drop(labels=['Unnamed: 0'],axis=1)

In [94]:
rppa_res

,measure_of_response,14-3-3_beta,14-3-3_epsilon_Caution,14-3-3_zeta,4E-BP1,4E-BP1_pS65,4E-BP1_pT37_T46,4E-BP1_pT70,53BP1,A-Raf_pS299_Caution,...,Tuberin_pT1462,VAV1_Caution,VEGFR2,VHL_Caution,XBP1_Caution,XRCC1_Caution,YAP_Caution,YAP_pS127_Caution,YB-1,YB-1_pS102
CELLLINE,,,,,,,,,,,,,,,,,,,,,
C2BBE1_LARGE_INTESTINE,Resistant,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,0.032598,-0.304864,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
C2BBE1_LARGE_INTESTINE,Resistant,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,0.032598,-0.304864,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
C2BBE1_LARGE_INTESTINE,Resistant,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,0.032598,-0.304864,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
C2BBE1_LARGE_INTESTINE,Sensitive,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,0.032598,-0.304864,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
C2BBE1_LARGE_INTESTINE,Resistant,0.142739,-0.012339,-0.526438,0.370901,0.087615,0.612442,-0.186115,0.032598,-0.304864,...,-0.325919,0.202650,1.155208,0.515055,-0.213274,-0.061394,0.870072,0.954783,-0.154237,0.252364
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NCIH2066_LUNG,Resistant,-0.130911,0.300023,-0.001885,0.130247,0.782626,0.403611,0.087359,0.259764,-0.317677,...,0.072654,-0.971477,-1.126916,0.595119,0.271774,-0.084982,-1.244111,-2.312100,0.333945,-0.046056
NCIH2066_LUNG,Resistant,-0.130911,0.300023,-0.001885,0.130247,0.782626,0.403611,0.087359,0.259764,-0.317677,...,0.072654,-0.971477,-1.126916,0.595119,0.271774,-0.084982,-1.244111,-2.312100,0.333945,-0.046056
NCIH2066_LUNG,Resistant,-0.130911,0.300023,-0.001885,0.130247,0.782626,0.403611,0.087359,0.259764,-0.317677,...,0.072654,-0.971477,-1.126916,0.595119,0.271774,-0.084982,-1.244111,-2.312100,0.333945,-0.046056


In [100]:
X = rppa_res.drop(['measure_of_response'], axis=1)
y = rppa_res['measure_of_response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [101]:
#Randomforest
clf = RandomForestClassifier(n_estimators = 500)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred), "\n")
print("\nAccuracy:", clf.score(X_test, y_test) * 100)

              precision    recall  f1-score   support

   Resistant       0.94      0.97      0.95       691
   Sensitive       0.96      0.92      0.94       561

    accuracy                           0.95      1252
   macro avg       0.95      0.94      0.94      1252
weighted avg       0.95      0.95      0.95      1252
 


Accuracy: 94.56869009584665


In [102]:
#SVM
clf = svm.SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))
print("\nAccuracy:", clf.score(X_test, y_test) * 100)

              precision    recall  f1-score   support

   Resistant       0.94      0.97      0.95       691
   Sensitive       0.96      0.93      0.94       561

    accuracy                           0.95      1252
   macro avg       0.95      0.95      0.95      1252
weighted avg       0.95      0.95      0.95      1252


Accuracy: 94.80830670926518


In [103]:
#XGboost
LE = LabelEncoder()
y_train = LE.fit_transform(y_train)
y_test = LE.fit_transform(y_test)

clf = XGBClassifier(eta = 0.5, max_leaf_nodes = 10)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred), "\n")
print("\nAccuracy:", clf.score(X_test, y_test) * 100)

              precision    recall  f1-score   support

           0       0.94      0.96      0.95       691
           1       0.95      0.93      0.94       561

    accuracy                           0.95      1252
   macro avg       0.95      0.95      0.95      1252
weighted avg       0.95      0.95      0.95      1252
 


Accuracy: 94.72843450479233
